In [3]:
import pandas as pd
import numpy as np
import json
import os
import requests

from gensim.utils import simple_preprocess

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils

In [5]:
path = './Lyrics_JSON'

In [6]:
# need to remove comments from lyrics

def gather_data(path_to_data):
    data = []
    
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-4:] == 'json':
                with open(os.path.join(path, f)) as t:
                    #for lyrics in t['Lyrics']:
                    text = t.read().strip('\n')
                    data.append(str(text))
                    
    return data

In [7]:
lyrics = gather_data(path)

In [13]:
# def tokenize(text):
#     return [token for token in simple_preprocess(text)]

In [15]:
with open('lyrics_aggregated.txt', 'w') as filehandle:
    filehandle.writelines('%s\n' % line for line in lyrics)

In [33]:
text = (open('lyrics_aggregated.txt').read())

In [34]:
text = text.lower()

In [37]:
text = text.replace('\n', '')
text = text.replace('{', '')
text = text.replace('}', '')
# text = text.replace("\", '')
text = text.replace('[', '')
text = text.replace(']', '')
text = text.replace('lyrics', '')
text = text.replace('title', '')
text = text.replace('"', '')
text = text.replace('genius', '')

In [45]:
# Creating character/word mappings
# All unique characters/words are mapped to a number

characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [47]:
# Training and target array for LSTM model

X = []
y = []

length = len(text)
seq_length = 100

for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    y.append(char_to_n[label])

In [49]:
# Modifying array shapes for LSTM, transform y into one-hot encoded

X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
y_modified = np_utils.to_categorical(y)

In [50]:
%%time

# Sequential model with two LSTM layers with 400 units each
# Dropoout layer to check for over-fitting

model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]),
              return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
CPU times: user 1.25 s, sys: 121 ms, total: 1.37 s
Wall time: 967 ms


In [51]:
%%time

# Baseline model

model.fit(X_modified, y_modified, epochs=1, batch_size=100)

model.save_weights('text_generator_400_0.2_400_0.2_baseline.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
44687/44687 [==============================] - 1821s 41ms/step - loss: 2.2785
CPU times: user 1h 37min 17s, sys: 7min 31s, total: 1h 44min 48s
Wall time: 30min 23s


In [92]:
model.load_weights('text_generator_400_0.2_400_0.2_baseline.h5')

In [107]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
#     string_mapped = string_mapped[1:len(string_mapped)]

ValueError: Error when checking input: expected lstm_1_input to have shape (100, 1) but got array with shape (101, 1)

In [98]:
# Combining text

txt = ''
for char in full_string:
    txt = txt + char
    
txt

'     and a prince and a big purple bear,            lives my favorite girl,            unaware of th                                                                                                                                                                                                                                                                                                                                                                                                                '

In [100]:
len(full_string)

500

In [104]:
string_mapped

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]